In [49]:
''' We are going to input the data from unemployment'''
import pandas as pd
data=pd.read_csv("../datasets/5.-Economics-&-Business/unemployment.csv")
pd.set_option('display.max_rows',1000)
data
from sqlalchemy import create_engine
import pymysql

In [2]:
dict = {'January':1,'February':2,"March":3,'April':4,'May':5,'June':6,'July':7,'August':8,'September':9,'October':10,'November':11,'December':12}
data.replace(dict,inplace=True)
data

,Year,Month,District Code,District Name,Neighborhood Code,Neighborhood Name,Gender,Demand_occupation,Number
0,2017,1,1,Ciutat Vella,1,el Raval,Male,Registered unemployed,2107
1,2017,1,1,Ciutat Vella,2,el Barri Gòtic,Male,Registered unemployed,538
2,2017,1,1,Ciutat Vella,3,la Barceloneta,Male,Registered unemployed,537
3,2017,1,1,Ciutat Vella,4,"Sant Pere, Santa Caterina i la Ribera",Male,Registered unemployed,741
4,2017,1,2,Eixample,5,el Fort Pienc,Male,Registered unemployed,630
...,...,...,...,...,...,...,...,...,...
14203,2013,12,10,Sant Martí,70,el Besòs i el Maresme,Female,Registered unemployed,1043
14204,2013,12,10,Sant Martí,71,Provençals del Poblenou,Female,Registered unemployed,738
14205,2013,12,10,Sant Martí,72,Sant Martí de Provençals,Female,Registered unemployed,1015
14206,2013,12,10,Sant Martí,73,la Verneda i la Pau,Female,Registered unemployed,1145


In [3]:
set(data.Month.tolist())

{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12}

### There's the distributions of unemployment between district in the many years

In [4]:
data_districts_year = data.groupby(['District Name','Year']).sum()
data_districts_year.drop( [ 'Month', 'District Code', 'Neighborhood Code'], axis = 1 ,inplace = True )
data_districts_year

Number
District Name       Year        
Ciutat Vella        2013  111245
                    2014  103594
                    2015  119131
                    2016  108314
                    2017  101100
Eixample            2013  185128
                    2014  169837
                    2015  190472
                    2016  169554
                    2017  156307
Gràcia              2013   89141
                    2014   81295
                    2015   91576
                    2016   81524
                    2017   74763
Horta-Guinardó      2013  145411
                    2014  135543
                    2015  150144
                    2016  134106
                    2017  122615
Les Corts           2013   52504
                    2014   49570
                    2015   56920
                    2016   50409
                    2017   45267
No consta           2013      54
                    2014      62
                    2015     131
                    2016     177
                    2017     245
Nou Barris          2013  177627
                    2014  167536
                    2015  185439
                    2016  165362
                    2017  151558
Sant Andreu         2013  134783
                    2014  126632
                    2015  143754
                    2016  126461
                    2017  113962
Sant Martí          2013  211972
                    2014  197409
                    2015  220456
                    2016  196603
                    2017  181531
Sants-Montjuïc      2013  158068
                    2014  150748
                    2015  169975
                    2016  150737
                    2017  137051
Sarrià-Sant Gervasi 2013   65280
                    2014   59365
                    2015   65740
                    2016   58837
                    2017   53710

# Find the worst district in Barcelona

In [5]:
data.groupby(['District Name']).sum()

,Year,Month,District Code,Neighborhood Code,Number
District Name,,,,,
Ciutat Vella,1547808,4992,768,1920,543384
Eixample,2321712,7488,2304,8640,871298
Gràcia,1934760,6240,5760,28800,418299
Horta-Guinardó,4256472,13728,14784,80256,687819
Les Corts,1160856,3744,2304,11520,254670
No consta,386952,1248,19008,19008,669
Nou Barris,5030376,16224,19968,124800,847522
Sant Andreu,2708664,8736,12096,80640,645592
Sant Martí,3869520,12480,19200,131520,1007971


In [15]:
# find the top in unemployment = Sant Marti




data_district = data.drop(['Month'], axis = 1)
district_max = data_district.groupby(['Year','District Name']).sum()

district_max_2017 = district_max.loc[2017]
district_max_2017.max()


District Code          4992
Neighborhood Code     32880
Number               181531
dtype: int64

# The first table : districts

In [23]:
data_districts = data.loc[:,['District Code','District Name']]
data_districts

,District Code,District Name
0,1,Ciutat Vella
1,1,Ciutat Vella
2,1,Ciutat Vella
3,1,Ciutat Vella
4,2,Eixample
...,...,...
14203,10,Sant Martí
14204,10,Sant Martí
14205,10,Sant Martí
14206,10,Sant Martí


# Create districts and insert table to SQL


In [67]:
districts = data_districts.drop_duplicates()
districts

,District Code,District Name
0,1,Ciutat Vella
4,2,Eixample
10,3,Sants-Montjuïc
18,4,Les Corts
21,5,Sarrià-Sant Gervasi
27,6,Gràcia
32,7,Horta-Guinardó
43,8,Nou Barris
56,9,Sant Andreu
63,10,Sant Martí


In [42]:
districts.index = districts['District Code']
districts.drop('District Code', axis=1, inplace=True)

/usr/local/lib/python3.7/site-packages/pandas/core/frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [43]:
districts

,District Name
District Code,
1,Ciutat Vella
2,Eixample
3,Sants-Montjuïc
4,Les Corts
5,Sarrià-Sant Gervasi
6,Gràcia
7,Horta-Guinardó
8,Nou Barris
9,Sant Andreu


In [85]:
driver = 'mysql+pymysql'
ip = '34.89.95.82'
username = 'Ironhack'
password = 'vladalex1234'
db = 'Project_2'
connection_string  = f'{driver}://{username}:{password}@{ip}/{db}'

engine = create_engine(connection_string)


In [55]:
districts.to_sql('Districts', engine)

# Insert the neighbor. to sql


In [81]:
neighborhoods = data.loc[:,['Neighborhood Code','Neighborhood Name']]

neighborhoods

,Neighborhood Code,Neighborhood Name
0,1,el Raval
1,2,el Barri Gòtic
2,3,la Barceloneta
3,4,"Sant Pere, Santa Caterina i la Ribera"
4,5,el Fort Pienc
...,...,...
14203,70,el Besòs i el Maresme
14204,71,Provençals del Poblenou
14205,72,Sant Martí de Provençals
14206,73,la Verneda i la Pau


In [84]:
neighborhoods.index = neighborhoods['Neighborhood Code']
neighborhoods.drop('Neighborhood Code', axis = 1, inplace = True)


In [86]:
neighborhoods.to_sql('Neighborhoods',engine)

# Drop the info in main table


In [63]:
data_good = data.drop(['District Name','Neighborhood Name'], axis = 1)
data_good

,Year,Month,District Code,Neighborhood Code,Gender,Demand_occupation,Number
0,2017,January,1,1,Male,Registered unemployed,2107
1,2017,January,1,2,Male,Registered unemployed,538
2,2017,January,1,3,Male,Registered unemployed,537
3,2017,January,1,4,Male,Registered unemployed,741
4,2017,January,2,5,Male,Registered unemployed,630
...,...,...,...,...,...,...,...
14203,2013,December,10,70,Female,Registered unemployed,1043
14204,2013,December,10,71,Female,Registered unemployed,738
14205,2013,December,10,72,Female,Registered unemployed,1015
14206,2013,December,10,73,Female,Registered unemployed,1145


In [64]:
driver = 'mysql+pymysql'
ip = '34.89.95.82'
username = 'Ironhack'
password = 'vladalex1234'
db = 'Project_2'
connection_string  = f'{driver}://{username}:{password}@{ip}/{db}'

engine = create_engine(connection_string)

In [65]:
data_good.to_sql('DataCore',engine)